In [3]:
#!pip install dagshub

In [8]:
import mlflow
from mlflow import MlflowClient
from steps import data_ingestion as di
from steps import data_transformation as dt
from steps import model_training as mt
import mlflow_exp as mle

In [1]:
import dagshub 
import os

DAGSHUB_TOKEN = dagshub.auth.get_token()

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=328987e8-e4cc-4727-a41a-27eea38393f3&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=94eed54018b82c621477658612922d06eaf8d3f1a602ebe3c30e63af7a29764f




Output()

In [6]:
import os

MLFLOW_TRACKING_URI='https://dagshub.com/Syamil-Ali/Loan_Application_ML.mlflow'
MLFLOW_TRACKING_USERNAME='Syamil-Ali'
MLFLOW_TRACKING_PASSWORD='2e6b27e2093c83eacbf6777913b6d087592d1710'

# It's recommended to define this within the code, because it's project specific (but this works too)
os.environ['MLFLOW_TRACKING_URI']=MLFLOW_TRACKING_URI

# Recommended to define as environment variables
os.environ['MLFLOW_TRACKING_USERNAME'] = MLFLOW_TRACKING_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = DAGSHUB_TOKEN

In [5]:
# create a client to access the MLflow tracking server
client = mlflow.MlflowClient()



In [7]:
# Create and experiment there

# MLFLOW SETUP

# CREATING EXPERIMENT


# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the application loan prediction project "
    "This experiment contains the models for loan application"
    "Also this experiment is for learning purpose"
)


# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "loan_application_project",
    "team": "syamil",
    "project_quarter": "Q3-2024",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_apples_experiment = client.create_experiment(
    name="Loan_Application_Models", tags=experiment_tags
)


In [9]:
# test

file_path = 'C:/Users/USER/Desktop/python project/MLFLOW/Real Loan Application/data/Applicant-details.csv'

# get file
df = di.data_ingest(file_path)

# cleaning and transform the dataset
X_train, y_train, X_test, y_test = dt.cleaning_train_pipeline(df)

params = {
    'max_iter': 100,
    'class_weight': 'balanced'
}


# train the model

model, score_train, score_valid, signature = mt.train_model(X_train, y_train, X_test, y_test, params)

In [15]:
# Log the result into mlflow


# setting up the param
exp_param = {
    'name' : 'Loan_Application_Models',
    'run_name': 'second_experiment',
    'artifact_path': 'loan_application_model',
    'model_name': 'Logistic Regression Dagshub', 
    'signature': signature
}

model_params = params
model_params['ways'] = 'balanced'

metrics = {
    'training_accuracy': score_train,
    'test_accuracy': score_valid
}

mle.mlflow_logging(exp_param, model, params, metrics)



Registered model 'Logistic Regression Dagshub' already exists. Creating a new version of this model...
2024/03/24 16:02:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Logistic Regression Dagshub, version 3
Created version '3' of model 'Logistic Regression Dagshub'.


In [11]:
# rename registered model

client.rename_registered_model(
    name = 'Logistic_Regressionv2',
    new_name = 'Logistic Regression Dagshub'
)

In [13]:
# transition the staging model from None to Staging

client.transition_model_version_stage(
    name='Logistic Regression Dagshub',
    version=1,
    stage="Staging"
)

<ModelVersion: creation_timestamp=1711263710367, current_stage='Staging', description='', last_updated_timestamp=1711264414129, name='Logistic Regression Dagshub', run_id='696fe8248eb943d7b5be8533446cda0f', run_link='', source='mlflow-artifacts:/7cbf54936da54dddb416f99b04b14240/696fe8248eb943d7b5be8533446cda0f/artifacts/loan_application_model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [26]:
for model in client.search_registered_models(filter_string="name LIKE '%'"):
    for model_version in model.latest_versions:
        if model_version.current_stage == 'Staging':
            deployment_model = model_version
        print(f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}")

name=Logistic Regression Dagshub; run_id=696fe8248eb943d7b5be8533446cda0f; version=1, stage=Staging
name=Logistic Regression Dagshub; run_id=fa43e67a689444e08761230c1a5b3645; version=3, stage=None


In [28]:
# model that will be use

#deployment_model.run_id

model_path = f'runs:/{deployment_model.run_id}/loan_application_model'

loaded_model = mlflow.pyfunc.load_model(model_path)

In [29]:
# test sample data

sample = X_test[1].reshape(-1, 11)


#loaded_model.predict(pd.DataFrame(X_test[0]))

In [31]:
loaded_model.predict(sample)

array([1], dtype=int64)